<a href="https://colab.research.google.com/github/srirambandi/ai/blob/master/examples/mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# when running in colab notebooks, first install library
!pip install import-ai
# upload respective dataset manually from examples directory of the library or download as below
!apt install subversion
!svn checkout https://github.com/srirambandi/ai/trunk/examples/MNIST

In [0]:
import ai
import numpy as np

In [0]:
def load_data(file):
    dict = np.load(file, allow_pickle=True)
    return dict

In [0]:
train_file = 'MNIST/train.npy'
test_file = 'MNIST/test.npy'

In [0]:
ai.manual_seed(2357)

In [0]:
class MNIST(ai.Module):
    def __init__(self):
        self.conv1 = ai.Conv2d(1, 8, kernel_size=3, stride=1)
        self.conv2 = ai.Conv2d(8, 16, kernel_size=3, stride=1)
        self.fc1 = ai.Linear(2304, 128)
        self.fc2 = ai.Linear(128, 10)
        
    def forward(self, x):
        o1 = ai.G.relu(self.conv1.forward(x))
        o2 = ai.G.relu(self.conv2.forward(o1))
        o3 = ai.G.dropout(ai.G.maxpool2d(o2), p=0.75)
        o4 = ai.G.dropout(ai.G.relu(self.fc1.forward(o3)), p=0.5)
        o5 = self.fc2.forward(o4)
        o6 = ai.G.softmax(o5)
        return o6

In [7]:
mnist = MNIST()
print(mnist)

MNIST(
  conv1: Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(0, 0), bias=True)
  conv2: Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(0, 0), bias=True)
  fc1: Linear(input_features=2304, output_features=128, bias=True)
  fc2: Linear(input_features=128, output_features=10, bias=True)
)


In [0]:
L = ai.Loss(loss_fn='CrossEntropyLoss')
optim = ai.Optimizer(mnist.parameters(), optim_fn='Adadelta', lr=1e-3)

In [0]:
train_dict = load_data(train_file)
inputs = train_dict.item()['data']
outputs = train_dict.item()['labels']
del train_dict

In [0]:
it, epoch = 0, 0
m = 128

In [0]:
def evaluate():
    ai.G.grad_mode = False
    file = test_file
    dict = load_data(file)
    inputs = dict.item()['data']
    outputs = dict.item()['labels']
    correct, total = 0, 0
    test_m = m
    for batch in range(int(len(outputs) / m)):
        input = inputs[batch * test_m : (batch + 1) * test_m].reshape(test_m, 1, 28, 28) / 255
        input =  np.stack([_ for _ in input], axis = -1)
        output = np.array(outputs[batch * test_m : (batch + 1) * test_m])
        scores = mnist.forward(input)
        preds = np.argmax(scores.data, axis=0)
        correct += np.sum(np.equal(output, preds))
        total += test_m
    accuracy = float(correct / total)
    ai.G.grad_mode = True
    return accuracy

In [12]:
while epoch < 15:
    epoch += 1
    it = 0
    for batch in range(int(len(outputs) / m)):
    # for batch in range(1):
        input = inputs[batch * m : (batch + 1) * m].reshape(m, 1, 28, 28) / 255
        input =  np.stack([_ for _ in input], axis = -1)
        output = outputs[batch * m : (batch + 1) * m]
        onehot = np.zeros((10, m))
        for _ in range(m):
            onehot[output[_], _] = 1.0
        scores = mnist.forward(input)
        loss = L.loss(scores, onehot)
        loss.backward()
        optim.step()        # update parameters with optimization functions
        optim.zero_grad()   # clearing the backprop list and resetting the gradients to zero
        if it%50 == 0:
            print('epoch: {}, iter: {}, loss: {}'.format(epoch, it, loss.data[0, 0]))
        it += 1
    print('Epoch {} completed. Accuracy: {:.2%} \n'.format(epoch, evaluate()))

using Adadelta
epoch: 1, iter: 0, loss: 2.3025840294914834
epoch: 1, iter: 50, loss: 0.9187123531440655
epoch: 1, iter: 100, loss: 0.4085635458226706
epoch: 1, iter: 150, loss: 0.6004088593765684
epoch: 1, iter: 200, loss: 0.29086178600654017
epoch: 1, iter: 250, loss: 0.34319691472121927
epoch: 1, iter: 300, loss: 0.21489279850395265
epoch: 1, iter: 350, loss: 0.26350197694647204
epoch: 1, iter: 400, loss: 0.33322564252865916
epoch: 1, iter: 450, loss: 0.22064459983074997
Epoch 1 completed. Accuracy: 97.06% 

epoch: 2, iter: 0, loss: 0.13708295377675725
epoch: 2, iter: 50, loss: 0.22588200376853215
epoch: 2, iter: 100, loss: 0.14417033101004634
epoch: 2, iter: 150, loss: 0.2001857327959781
epoch: 2, iter: 200, loss: 0.23969054558464972
epoch: 2, iter: 250, loss: 0.17138288785463435
epoch: 2, iter: 300, loss: 0.2069050755448711
epoch: 2, iter: 350, loss: 0.19709628931058948
epoch: 2, iter: 400, loss: 0.26908922200806024
epoch: 2, iter: 450, loss: 0.22399071828882627
Epoch 2 completed. 

In [13]:
mnist.save()

saving model...
Successfully saved model in MNIST.npy
